In [1]:
import numpy as np
import pandas as pd
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
stack_1 = pd.read_pickle('stack/stack_1.pkl')[0].values
stack_2 = pd.read_pickle('stack/stack_2.pkl')[0].values
stack_3 = pd.read_pickle('stack/stack_3.pkl')[0].values

In [3]:
stack = []
for i in range(stack_1.shape[0]):
    s1, s2, s3 = stack_1[i], stack_2[i], stack_3[i]
    if s1 != -1: stack.append(s1)
    if s2 != -1: stack.append(s2)
    if s3 != -1: stack.append(s3)

In [4]:
stack_arr = np.array(stack)

In [5]:
result_1 = pd.read_pickle('stack/result_1.pkl')['category'].values
result_2 = pd.read_pickle('stack/result_2.pkl')['category'].values
result_3 = pd.read_pickle('stack/result_3.pkl')['category'].values

In [6]:
result = []
for i in range(result_1.shape[0]):
    sum = result_1[i] + result_2[i] + result_3[i]
    if sum > 1: result.append(1)
    else: result.append(0)
result_arr = np.array(result)

In [7]:
data_set = np.loadtxt("dataset/training_set.txt").astype(np.int64)
data = data_set[:, :2]
y = data_set[:, -1]
test = np.loadtxt("dataset/testing_set.txt").astype(np.int64)

In [150]:
features = pd.read_pickle('features.pkl')
features['id_differ']= [data[i,0]-data[i,1] for i in range(data_set.shape[0])]
features['rnn'] = stack_arr

In [157]:
lsa1 = pd.read_pickle('lsa1.pkl')
lsa2 = pd.read_pickle('lsa2.pkl')

In [158]:
X = np.concatenate((features.values, lsa1.values[:,:15], lsa2.values[:,:15]), axis=1)

In [159]:
seed = 7
test_size = 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
X_train.shape

In [161]:
model = XGBClassifier(max_depth=12, n_jobs = 4, objective='binary:logistic',n_estimators = 300)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=12, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=4, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [162]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [163]:
accuracy = accuracy_score(y_test, predictions)
f1= f1_score(y_test, predictions)
print("Accuracy: %.3f%%" % (accuracy * 100.0))
print("F1score: %.3f%%" % (accuracy * 100.0))

Accuracy: 98.155%
F1score: 98.155%


In [20]:
features_test = pd.read_pickle('features_test.pkl')
features_test['id_differ']= [test[i,0]-test[i,1] for i in range(test.shape[0])]
features_test['rnn'] = result_arr

In [26]:
lsa_test1 = pd.read_pickle('lsa_test1.pkl')
lsa_test2 = pd.read_pickle('lsa_test2.pkl')
X = np.concatenate((features_test.values, lsa_test1.values[:,:15], lsa_test2.values[:,:15]), axis=1)

In [27]:
y_pred = model.predict(X)
predictions = [round(value) for value in y_pred]

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [28]:
pred_dict = {idx: predictions[idx] for idx in range(len(predictions))}
submission = pd.DataFrame.from_dict(pred_dict,orient='index')
submission.index.names = ['id']
submission.columns = ['category']
submission.to_csv("submission.csv")